# Draw an isochrone map with OSMnx

Author: [Geoff Boeing](https://geoffboeing.com/)

How far can you travel on foot in 15 minutes?

  - [Overview of OSMnx](http://geoffboeing.com/2016/11/osmnx-python-street-networks/)
  - [GitHub repo](https://github.com/gboeing/osmnx)
  - [Examples, demos, tutorials](https://github.com/gboeing/osmnx-examples)
  - [Documentation](https://osmnx.readthedocs.io/en/stable/)
  - [Journal article/citation](http://geoffboeing.com/publications/osmnx-complex-street-networks/)

In [1]:
import geopandas as gpd
import matplotlib.pyplot as plt
import networkx as nx
import osmnx as ox
from descartes import PolygonPatch
from shapely.geometry import Point, LineString, Polygon
import shapely
from shapely import MultiPolygon

In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle

In [3]:
import warnings
warnings.simplefilter("ignore")

In [4]:
# configure the place, network type, trip times, and travel speed
network_type = 'walk'
trip_times = [5, 10, 15, 20] #in minutes
travel_speed = 4 #walking speed in km/hour

In [5]:
# get one color for each isochrone
iso_colors = ox.plot.get_colors(n=len(trip_times), cmap='plasma', start=0)

## Getting data

In [6]:
metro_df = pd.read_csv('data/metro-exits-2024-12-18.csv', delimiter=';')
del metro_df['Unnamed: 21']
metro_df.head()

,Локальный идентификатор,Наименование,На территории Москвы,Административный округ,Район,Долгота в WGS-84,Широта в WGS-84,Тип вестибюля,Станция метрополитена,Линия,...,Режим работы по чётным дням,Режим работы по нечётным дням,Количество полнофункциональных БПА (все типы билетов),Количество малофункциональных БПА (билеты на 1 и 2 поездки),Общее количество БПА,Ремонт эскалаторов,Статус объекта,global_id,geoData,geodata_center
0,331,"Китай-город, вход-выход 5 в северный вестибюль",да,Центральный административный округ,Басманный район,37.631765,55.757328,подземный,Китай-город,Калужско-Рижская линия,...,открытие в 05:30:00; закрытие в 01:00:00; перв...,открытие в 05:30:00; закрытие в 01:00:00; перв...,NaN,4.0,4.0,nested data,действует,1773539,"{coordinates=[37.63176509, 55.75732811], type=...","{coordinates=[37.63176509, 55.75732811], type=..."
1,327,"Китай-город, вход-выход 4 в северный вестибюль",да,Центральный административный округ,Тверской район,37.630924,55.756733,подземный,Китай-город,Калужско-Рижская линия,...,открытие в 05:30:00; закрытие в 01:00:00; перв...,открытие в 05:30:00; закрытие в 01:00:00; перв...,NaN,4.0,4.0,nested data,действует,1773540,"{coordinates=[37.63092407, 55.75673268], type=...","{coordinates=[37.63092407, 55.75673268], type=..."
2,330,"Китай-город, вход-выход 7 в северный вестибюль",да,Центральный административный округ,Басманный район,37.631862,55.757027,подземный,Китай-город,Калужско-Рижская линия,...,открытие в 05:30:00; закрытие в 01:00:00; перв...,открытие в 05:30:00; закрытие в 01:00:00; перв...,NaN,4.0,4.0,nested data,действует,1773541,"{coordinates=[37.63186197, 55.75702717], type=...","{coordinates=[37.63186197, 55.75702717], type=..."
3,322,"Китай-город, вход-выход 13 в южный вестибюль",да,Центральный административный округ,Тверской район,37.633207,55.752999,подземный,Китай-город,Калужско-Рижская линия,...,открытие в 05:30:00; закрытие в 01:00:00; перв...,открытие в 05:30:00; закрытие в 01:00:00; перв...,NaN,4.0,4.0,nested data,действует,1773542,"{coordinates=[37.63320674, 55.7529987], type=P...","{coordinates=[37.63320674, 55.7529987], type=P..."
4,321,"Китай-город, вход-выход 12 в южный вестибюль",да,Центральный административный округ,Таганский район,37.633566,55.753071,подземный,Китай-город,Калужско-Рижская линия,...,открытие в 05:30:00; закрытие в 01:00:00; перв...,открытие в 05:30:00; закрытие в 01:00:00; перв...,NaN,4.0,4.0,nested data,действует,1773543,"{coordinates=[37.63356616, 55.75307131], type=...","{coordinates=[37.63356616, 55.75307131], type=..."


In [7]:
metro_df = metro_df[metro_df['Линия'] != 'Московская монорельсовая транспортная система']

In [195]:
metro_df['Линия'].value_counts()

Линия
Таганско-Краснопресненская линия    124
Сокольническая линия                122
Калужско-Рижская линия              117
Серпуховско-Тимирязевская линия     117
Замоскворецкая линия                108
Люблинско-Дмитровская линия          99
Большая кольцевая линия              97
Арбатско-Покровская линия            78
Московское центральное кольцо        73
Солнцевская линия                    50
Калининская линия                    39
Некрасовская линия                   32
Кольцевая линия                      30
Филёвская линия                      27
Бутовская линия Лёгкого метро        11
Троицкая линия                       10
Name: count, dtype: int64

In [8]:
metro_df = metro_df.iloc[:, [5, 6, 8, 9]]
metro_df.head()

,Долгота в WGS-84,Широта в WGS-84,Станция метрополитена,Линия
0,37.631765,55.757328,Китай-город,Калужско-Рижская линия
1,37.630924,55.756733,Китай-город,Калужско-Рижская линия
2,37.631862,55.757027,Китай-город,Калужско-Рижская линия
3,37.633207,55.752999,Китай-город,Калужско-Рижская линия
4,37.633566,55.753071,Китай-город,Калужско-Рижская линия


In [152]:
metro_df[metro_df['Станция метрополитена'] == 'Мнёвники'].iloc[:, [0, 1]].mean()

Долгота в WGS-84    37.472022
Широта в WGS-84     55.761102
dtype: float64

In [153]:
metro_df[metro_df['Станция метрополитена'] == 'Мнёвники'].iloc[:, [0, 1]].to_numpy()

array([[37.47138012, 55.76026171],
       [37.47172881, 55.76017096],
       [37.47233066, 55.76200533],
       [37.47264716, 55.76196903]])

In [43]:
metro_stations = list(set([x[0] + '_' + x[1] for x in metro_df.iloc[:, [2, 3]].values.tolist()]))
metro_stations[:5]

['Давыдково_Большая кольцевая линия',
 'Волоколамская_Арбатско-Покровская линия',
 'Третьяковская_Калининская линия',
 'Мичуринский проспект_Большая кольцевая линия',
 'Войковская_Замоскворецкая линия']

In [44]:
len(metro_stations)

295

In [66]:
metro_df.rename(columns={'Станция метрополитена': 'Station', 'Линия': 'Line', 
                         'Долгота в WGS-84': 'Longitude', 'Широта в WGS-84': 'Latitude'}, inplace=True)

In [64]:
train_df = pd.read_csv('data/train-exits-2024-12-02.csv', delimiter=';')
train_df

,Локальный идентификатор,Наименование,Станция радиального железнодорожного направления,Радиальное железнодорожное направление,Режим работы,Широта в WGS-84,Долгота в WGS-84,Статус объекта,global_id,geoData,geodata_center,Unnamed: 11
0,2,Ленинградский вокзал (проход 1),Ленинградский вокзал,Ленинградское,круглосуточно,55.777311,37.655229,действует,1272821225,"{coordinates=[37.655229, 55.777311], type=Point}",NaN,NaN
1,31,Лихоборы (проход 1),Лихоборы,Ленинградское,05:30-01:00,55.848527,37.552228,действует,1609179357,"{coordinates=[37.552228, 55.848527], type=Point}",NaN,NaN
2,200,Фили (проход 1),Фили,Белорусское,05:30-01:00,55.744283,37.514404,действует,1609205182,"{coordinates=[37.514404, 55.744283], type=Point}",NaN,NaN
3,338,Щукинская (проход 1),Щукинская,Рижское,05:30-01:00,55.810905,37.461462,действует,1609205184,"{coordinates=[37.461462, 55.810905], type=Point}",NaN,NaN
4,313,Стрешнево (проход 4),Стрешнево,Рижское,05:30-01:00,55.815014,37.487330,действует,1609205185,"{coordinates=[37.48733, 55.815014], type=Point}",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
271,548,Курская (проход 5),Курская,Горьковское,05:30-01:00,55.758704,37.660939,действует,2696710407,"{coordinates=[37.660939, 55.758704], type=Point}",NaN,NaN
272,549,Остафьево (проход 1),Остафьево,Курское,05:30-01:00,55.485849,37.555761,действует,2696952123,"{coordinates=[37.555761, 55.485849], type=Point}",NaN,NaN
273,550,Остафьево (проход 2),Остафьево,Курское,05:30-01:00,55.486068,37.554731,действует,2696952136,"{coordinates=[37.554731, 55.486068], type=Point}",NaN,NaN
274,551,Мещерская (проход 1),Мещерская,Киевское,05:30-01:00,55.667150,37.425124,действует,2696952180,"{coordinates=[37.425124, 55.66715], type=Point}",NaN,NaN


In [65]:
train_df = train_df.iloc[:, [2, 3, 5, 6]]
train_df.head()

,Станция радиального железнодорожного направления,Радиальное железнодорожное направление,Широта в WGS-84,Долгота в WGS-84
0,Ленинградский вокзал,Ленинградское,55.777311,37.655229
1,Лихоборы,Ленинградское,55.848527,37.552228
2,Фили,Белорусское,55.744283,37.514404
3,Щукинская,Рижское,55.810905,37.461462
4,Стрешнево,Рижское,55.815014,37.487330


In [41]:
train_stations = list(set([x[0] + '_' + x[1] for x in train_df.iloc[:, [0, 1]].values.tolist()]))
train_stations[:5]

['Савёловская_Савёловское',
 'Печатники_Курское',
 'Серп и Молот_Горьковское',
 'Трикотажная_Рижское',
 'Беговая_Белорусское']

In [42]:
len(train_stations)

112

In [68]:
train_df.rename(columns={'Станция радиального железнодорожного направления': 'Station', 'Радиальное железнодорожное направление': 'Line', 
                         'Долгота в WGS-84': 'Longitude', 'Широта в WGS-84': 'Latitude'}, inplace=True)

## Download and prep the street network

In [116]:
METRO_MARGIN = 500 # m
TRAIN_MARGIN = 400 # m
CRS = "EPSG:4326"

def get_graph_and_nodes(station, df, trip_times, margin):
    st_name, st_line = list(station.split('_'))
    station_df = df[(df.Station == st_name) & (df.Line == st_line)].loc[:, ['Longitude', 'Latitude']]
    coords = station_df.mean()
    station_coords = coords[1], coords[0]

    meters_per_minute = travel_speed * 1000 / 60 # km per hour to m per minute
    G = ox.graph_from_point(station_coords, dist=meters_per_minute * max(trip_times) + margin, dist_type='network',
                            network_type=network_type, simplify=False)

    gdf_nodes = ox.graph_to_gdfs(G, edges=False)

    exits = station_df.to_numpy()

    G = ox.project_graph(G, to_crs=CRS)
    exit_nodes = [ox.distance.nearest_nodes(G, exit[0], exit[1]) for exit in exits]
    G = ox.project_graph(G)

    for u, v, k, data in G.edges(data=True, keys=True):
        data['time'] = data['length'] / meters_per_minute

    return G, exit_nodes

In [46]:
def make_iso_polys(G, center_node, edge_buff=25, node_buff=50, infill=False):
    isochrone_polys = []
    for trip_time in sorted(trip_times, reverse=True):
        subgraph = nx.ego_graph(G, center_node, radius=trip_time, distance='time')

        node_points = [Point((data['x'], data['y'])) for node, data in subgraph.nodes(data=True)]
        nodes_gdf = gpd.GeoDataFrame({'id': subgraph.nodes()}, geometry=node_points)
        nodes_gdf = nodes_gdf.set_index('id')

        edge_lines = []
        for n_fr, n_to in subgraph.edges():
            f = nodes_gdf.loc[n_fr].geometry
            t = nodes_gdf.loc[n_to].geometry
            edge_lookup = G.get_edge_data(n_fr, n_to)[0].get('geometry',  LineString([f,t]))
            edge_lines.append(edge_lookup)

        n = nodes_gdf.buffer(node_buff).geometry
        e = gpd.GeoSeries(edge_lines).buffer(edge_buff).geometry
        all_gs = list(n) + list(e)
        new_iso = gpd.GeoSeries(all_gs).union_all()

        g_node = G.nodes(data=True)[center_node]
        if type(new_iso) != Polygon:
            new_iso = Polygon([(g_node['x'], g_node['y'])]*4)

        # try to fill in surrounded areas so shapes will appear solid and blocks without white space inside them
        if infill:
            new_iso = Polygon(new_iso.exterior)
        isochrone_polys.append(new_iso)
    return isochrone_polys

In [99]:
def show_isochrones(G, isochrones, iso_colors):
    fig, ax = ox.plot_graph(G, show=False, close=False, edge_color='#999999', edge_alpha=0.2,
                                node_size=0, bgcolor='k')
    
    for polygon, fc in zip(isochrones, iso_colors):
        if type(polygon) != MultiPolygon:
            polygon = MultiPolygon([polygon])
        
        for geom in polygon.geoms:    
            xs, ys = geom.exterior.xy 
            ax.fill(xs, ys, fc=fc, ec='none', alpha=0.6, zorder=-1)
    
    plt.show()

In [58]:
def plot_mapa(mapa):
    fig, axs = plt.subplots()
    axs.set_aspect('equal', 'datalim')
    
    for polygon, fc in zip(mapa, iso_colors):
        for geom in polygon.geoms:    
            xs, ys = geom.exterior.xy 
            axs.fill(xs, ys, fc=fc, ec='none', alpha=0.6, zorder=-1)
    
    plt.show()

In [117]:
with open('data/metro_isochrones.pickle', 'rb') as f:
    metro_station_to_isochrones = pickle.load(f)

In [ ]:
for station in tqdm(metro_stations[:]):
    print(f'================= {station.replace("_", ", ")} =================')

    if station not in metro_station_to_isochrones.keys():
        G, exit_nodes = get_graph_and_nodes(station, metro_df, trip_times, METRO_MARGIN)
        
        exits_isochrones = []
        for exit_node in exit_nodes:
            isochrone_polys = make_iso_polys(G, exit_node, edge_buff=25, node_buff=0, infill=True)
            # print(isochrone_polys)
            exits_isochrones.append(isochrone_polys)
            # show_isochrones(G, isochrone_polys, iso_colors)
    
        station_isochrones = []
        for i in range(len(trip_times)):
            iso = shapely.union_all([iso_polys[i] for iso_polys in exits_isochrones])
            if type(iso) != MultiPolygon:
                iso = MultiPolygon([iso])
            station_isochrones.append(iso)
        # print(station_isochrones)

        metro_station_to_isochrones[station] = station_isochrones
        with open('data/metro_isochrones.pickle', 'wb') as f:
             pickle.dump(metro_station_to_isochrones, f)

        # show_isochrones(G, station_isochrones, iso_colors)
    else:
        print(metro_station_to_isochrones[station])

 11%|████████▋                                                                       | 32/295 [00:00<00:01, 151.30it/s]

================= Давыдково, Большая кольцевая линия =================
[<MULTIPOLYGON (((401850.082 6175085.601, 401847.639 6175085.38, 401845.185 6...>, <MULTIPOLYGON (((402186.003 6175266.959, 402185.637 6175267.091, 402185.263 ...>, <MULTIPOLYGON (((402320.716 6175333.251, 402318.395 6175334.046, 402316.163 ...>, <MULTIPOLYGON (((402606.404 6175345.996, 402603.985 6175345.59, 402601.537 6...>]
================= Волоколамская, Арбатско-Покровская линия =================
[<MULTIPOLYGON (((397730.948 6188957.229, 397731.442 6188956.741, 397731.731 ...>, <MULTIPOLYGON (((398018.395 6188969.534, 398019.094 6188969.015, 398019.807 ...>, <MULTIPOLYGON (((398317.503 6188671.206, 398308.108 6188667.83, 398275.694 6...>, <MULTIPOLYGON (((398582.439 6188688.919, 398581.835 6188688.891, 398581.213 ...>]
================= Третьяковская, Калининская линия =================
[<MULTIPOLYGON (((412806.341 6177579.861, 412803.89 6177579.756, 412801.44 61...>, <MULTIPOLYGON (((412979.015 6177743.59, 41

 11%|████████▋                                                                       | 32/295 [00:17<00:01, 151.30it/s]

In [ ]:
metro_mapa = [shapely.union_all([x[i] for x in list(metro_station_to_isochrones.values())]) for i in range(len(trip_times))]
metro_mapa

In [ ]:
plot_mapa(metro_mapa)

In [ ]:
with open('data/train_isochrones.pickle', 'rb') as f:
    train_station_to_isochrones = pickle.load(f)

In [ ]:
for station in tqdm(train_stations[:]):
    print(f'================= {station.replace("_", ", ")} =================')

    if station not in train_station_to_isochrones.keys():
        G, exit_nodes = get_graph_and_nodes(station, train_df, trip_times, TRAIN_MARGIN)
        
        exits_isochrones = []
        for exit_node in exit_nodes:
            isochrone_polys = make_iso_polys(G, exit_node, edge_buff=25, node_buff=0, infill=True)
            # print(isochrone_polys)
            exits_isochrones.append(isochrone_polys)
            # show_isochrones(G, isochrone_polys, iso_colors)
    
        station_isochrones = []
        for i in range(len(trip_times)):
            iso = shapely.union_all([iso_polys[i] for iso_polys in exits_isochrones])
            if type(iso) != MultiPolygon:
                iso = MultiPolygon([iso])
            station_isochrones.append(iso)
        print(station_isochrones)

        train_station_to_isochrones[station] = station_isochrones
        with open('data/train_isochrones.pickle', 'wb') as f:
             pickle.dump(train_station_to_isochrones, f)

        # show_isochrones(G, station_isochrones, iso_colors)
    else:
        print(train_station_to_isochrones[station])

In [ ]:
train_mapa = [shapely.union_all([x[i] for x in list(train_station_to_isochrones.values())]) for i in range(len(trip_times))]
train_mapa

In [ ]:
plot_mapa(train_mapa)

In [ ]:
mapa = [shapely.union_all([metro_mapa[i], train_mapa[i]]) for i in range(len(trip_times))]

In [ ]:
plot_mapa(mapa)